## Predicting analogy type from story pairs

In [1]:
import pandas
from sklearn.model_selection import train_test_split

data = pandas.read_csv("C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/aesop_morals_pairs.csv")
data = data[0:44]

del data["Unnamed: 2"]
del data["ID"]
data_2 = pandas.read_csv("C:/Users/Thiloshon/PycharmProjects/analogical-transfer-learning/aesop_morals_com_commontags - edit - hodonometer.csv")

st1 = []
for st in data["STORY 1"]:
    st1.append(data_2[data_2["story"]==st]["frames"].values[0])
    
st2 = []
for st in data["STORY 2"]:
    st2.append(data_2[data_2["story"]==st]["frames"].values[0])
    
data["FRAME 1"] = st1
data["FRAME 2"] = st2


train, test = train_test_split(data, test_size=0.20)


In [2]:
data

,STORY 1,STORY 2,Shallow attribute analogy (SAA),Deep attribute analogy (DAA),Relational analogy (RA),Situational analogy (SA),Moral/purpose (MP),Literal similarity (LS),STORY SHAPE SIMILARITY (SSS),SAA Score,DAA Score,RA Score,SA Score,MP Score,LS Score,Arc Score,TOTAL,FRAME 1,FRAME 2
0,THE EAGLE and the Beetle were at enmity togeth...,"THE FROGS, grieved at having no established Ru...",No,No,Yes,No,No,Yes,Yes,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,Travel Collaboration Biological_area Activity_...,Age Possession Text Perception_experience Soun...
1,"SOME DOGS, finding the skin of a lion, began t...",A DOG complained of the cruelty of her master ...,No,Yes,No,No,No,Yes,Yes,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,Arriving Buildings Possession Arriving Biologi...,Bringing Natural_features Cardinal_numbers Inc...
2,"A STAG saw his shadow reflected in the water, ...","A HOUND, who in the days of his youth and stre...",No,Yes,Yes,Yes,No,No,Yes,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,Collaboration Destroying Increment Giving Ordi...,Beat_opponent Be_subset_of Fields Animals Dese...
3,"THE ASS and the Fox, having entered into partn...","A STAG saw his shadow reflected in the water, ...",No,No,No,No,No,No,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Arriving Motion Locative_relation Biological_a...,Collaboration Destroying Increment Giving Ordi...
4,"A MOUSE, by an unlucky chance, formed an intim...","A GARDENER, who lived alone, became discontent...",No,Yes,Yes,No,Yes,No,No,0.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,Biological_area Desirability Posture Self_moti...,Posture Calendric_unit Calendric_unit Non-grad...
5,A WILD ASS and a Lion entered into an alliance...,A THIEF came in the night to break into a hous...,No,No,No,No,No,No,Yes,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,Expansion Judgment Process_end Statement Posse...,Arriving Calendric_unit Buildings Desirable_ev...
6,A VERY SKILLFUL BOWMAN went to the mountains i...,"A STAG saw his shadow reflected in the water, ...",No,No,Yes,No,No,Yes,No,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,Fleeing Buildings Request Statement Killing Co...,Collaboration Destroying Increment Giving Ordi...
7,THE ANTS were employing a fine winter's day in...,A PRINCE had some Monkeys trained to dance. Be...,No,No,No,No,No,No,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Desiring Protecting Desiring Natural_features ...,Creating Participation Motion Size Process_end...
8,A RICH NOBLEMAN once opened the theater to the...,"THE ASS and the Fox, having entered into partn...",No,Yes,Yes,Yes,No,No,Yes,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,Residence Kinship Difficulty Telling Theft Cal...,Arriving Motion Locative_relation Biological_a...
9,"A NIGHTINGALE, sitting aloft upon an oak, was ...","A HOUND, having started a Hare on the hill-sid...",No,No,No,No,No,No,Yes,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,Posture Perception_experience Change_position_...,Kinship Spatial_contact Means Come_together Pe...


In [63]:
# +vs class distribution

a = list(train["RA Score"])
print(test["RA Score"])

sum(a)/len(a)

2     1.0
10    1.0
9     0.0
24    1.0
8     1.0
16    0.0
21    1.0
14    0.0
31    0.0
Name: RA Score, dtype: float64


0.5142857142857142

In [55]:
# Cross encoder classifer

from sentence_transformers import InputExample
from sentence_transformers import models, losses, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.cross_encoder.evaluation import CEBinaryAccuracyEvaluator
import math
import matplotlib.pyplot as plt
from sentence_transformers import CrossEncoder
from sklearn.metrics import accuracy_score

train_samples = [InputExample(texts=[a[0], a[1]], label=int(a[2])) for a in zip(train['STORY 1'], train['STORY 2'], train['RA Score'])] 

model = CrossEncoder('distilroberta-base', num_labels=1)

train_dataloader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=4)

num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8 [00:00<?, ?it/s]

In [59]:
# additional training

history = model.fit(train_dataloader  = train_dataloader,
                    epochs=num_epochs,
                    evaluation_steps=int(len(train_dataloader)*0.1),
                    warmup_steps=warmup_steps,
                    use_amp=False,
                   )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8 [00:00<?, ?it/s]

In [61]:
# Test prediction

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

test_samples = [(a[0], a[1]) for a in zip(test['STORY 1'], test['STORY 2'])] 

a = model.predict(test_samples)
preds = [int(ap>0.5) for ap in a]

print(accuracy_score(test["RA Score"], preds))
print(f1_score( test["RA Score"], preds, average="macro"))

0.5555555555555556
0.35714285714285715


In [62]:
print(test["RA Score"])
print(preds)

2     1.0
10    1.0
9     0.0
24    1.0
8     1.0
16    0.0
21    1.0
14    0.0
31    0.0
Name: RA Score, dtype: float64
[1, 1, 1, 1, 1, 1, 1, 1, 1]


In [72]:
# Sample story
print(test["STORY 1"][24])
test["STORY 2"][24]

A porcupine, wanting to shelter himself, desired a nest of Snakes to give him admittance into their cave. They were prevailed upon, and let him in accordingly; but were so annoyed with his sharp prickly quills that they soon repented of their easy compliance, and entreated the Porcupine to withdraw, and leave them their hole to themselves. No, says he, let them quit the place that don't like it; for my part, I am well enough satisfied as I am.


'AN EAGLE, overwhelmed with sorrow, sat upon the branches of a tree, in company with a Kite. Why, said the Kite: do I see you with such a rueful look? I seek, she replied: for a mate suitable for me, and am not able to find one. Take me, returned the Kite: I am much stronger than you are. Why, are you able to secure the means of living by your plunder? Well, I have often caught and carried away an ostrich in my talons. The Eagle, persuaded by these words, accepted him as her mate. Shortly after the nuptials, the Eagle said: Fly off and bring me back the ostrich you promised me. The Kite, soaring aloft into the air, brought back the shabbiest possible mouse. Is this, said the Eagle: the faithful fulfillment of your promise to me? The Kite replied: That I might attain your royal hand, there is nothing that I would not have promised, however much I knew that I must fail in the performance.'